<a href="https://colab.research.google.com/github/zhr-frj/daftarchePhd/blob/main/Copy_of_Untitled0LaBSEGemma9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Nov 14 21:28:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# --- نصب استاندارد و پایدار ---
print("--- در حال نصب کتابخانه‌های استاندارد ---")
!pip install langchain langchain-community transformers bitsandbytes accelerate faiss-cpu pypdf sentence-transformers langchain_huggingface docarray -q
print("--- نصب کامل شد ---")

--- در حال نصب کتابخانه‌های استاندارد ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
--- نصب کامل شد ---


In [5]:
#  --- وارد کردن ابزارها و تنظیمات فشرده‌سازی ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

print("--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---")
# تنظیمات فشرده‌سازی 4-bit برای کاهش مصرف حافظه
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)
print("--- ابزارها و تنظیمات آماده شدند ---")

--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---
--- ابزارها و تنظیمات آماده شدند ---


In [6]:
# --- بارگذاری «مترجم» (Tokenizer) - Gemma 9B ---
from transformers import AutoTokenizer
# --- بارگذاری «مترجم» (Tokenizer) ---
print("--- در حال بارگذاری «مترجم» (Tokenizer) ---")
tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-9b-it", use_fast=True)
print("--- «مترجم» (Tokenizer) بارگذاری شد ---")

--- در حال بارگذاری «مترجم» (Tokenizer) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

--- «مترجم» (Tokenizer) بارگذاری شد ---


In [7]:
# --- بارگذاری «مغز» (LLM) ---

from transformers import AutoModelForCausalLM
print("--- در حال بارگذاری «مغز» (LLM) --- (این مرحله بسیار سنگین و زمان‌بر است)")
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2-9b-it",
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)
print("--- «مغز» (LLM) با موفقیت بارگذاری شد ---")

--- در حال بارگذاری «مغز» (LLM) --- (این مرحله بسیار سنگین و زمان‌بر است)


config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- «مغز» (LLM) با موفقیت بارگذاری شد ---


In [32]:
# --- (نهایی): ساخت «خط لوله» (Pipeline) و ابزارهای LangChain ---
print("--- در حال ساخت «خط لوله» (Pipeline) ---")

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # افزایش طول پاسخ
    return_full_text=False # --- تغییر کلیدی: فقط جواب را برگردان، نه پرامپت را ---
)

# ساخت «غلاف» LangChain برای LLM
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# ساخت «تمیزکننده خروجی»
parser = StrOutputParser()

print("--- «خط لوله»، «غلاف» و «تمیزکننده» (نسخه نهایی) آماده شدند ---")

Device set to use cuda:0


--- در حال ساخت «خط لوله» (Pipeline) ---
--- «خط لوله»، «غلاف» و «تمیزکننده» (نسخه نهایی) آماده شدند ---


In [33]:
# --- بارگذاری «بینی معنی‌فهم» (Embedding Model) - نسخه چندزبانه LaBSE ---

from langchain_huggingface import HuggingFaceEmbeddings

print("--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) - نسخه قوی چندزبانه (LaBSE) ---")

# --- تغییر کلیدی ---
# استفاده از مدل قدرتمند و چندزبانه LaBSE
embedding_name = 'sentence-transformers/LaBSE'
# --- پایان تغییر ---

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name,
    model_kwargs={'device': 'cpu'} # همچنان روی CPU اجرا می‌کنیم تا GPU آزاد بماند
)
print("--- «بینی معنی‌فهم» (Embeddings) جدید آماده شد ---")

--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) - نسخه قوی چندزبانه (LaBSE) ---
--- «بینی معنی‌فهم» (Embeddings) جدید آماده شد ---


In [76]:
# --- تعریف «قالب چت» (ChatPromptTemplate) - نسخه ترکیبی نهایی ---
from langchain_core.prompts import ChatPromptTemplate

print("--- 🤖 در حال تعریف «قالب چت» (ChatPromptTemplate) - نسخه ترکیبی نهایی ---")

# --- تغییر کلیدی ---
# ما هر دو دستورالعمل موفق را ترکیب می‌کنیم:
# ۱. دستور «گردآوری» (برای حل مشکل سهمیه)
# ۲. دستور «دقت به ساختار» (برای حل مشکل زبان فارسی)
SYSTEM_TEMPLATE = """
شما یک دستیار هوشمند، دقیق و مفید هستید.
وظیفه شما پاسخ به «سوال کاربر» با استفاده از اطلاعات موجود در «زمینه» است.

*دستورالعمل‌های کلیدی:*
۱. *گردآوری کامل (Synthesize):* پاسخ شما باید *خلاصه‌ای کامل* از *تمام* اطلاعات مرتبطی باشد که در «زمینه» در مورد «سوال کاربر» پیدا می‌کنید. (مثلاً اگر دو سهمیه مختلف ذکر شده، هر دو را بیاورید).
۲. *دقت به ساختار (Structure-Aware):* به ساختار متن، جداول، سرفصل‌ها و لیست‌ها دقت کنید. مطمئن شوید که اطلاعات را از بخش درستی استخراج می‌کنید (مثلاً «گرایش‌های رشته» را با «دروس امتحانی» یا «رشته‌های مرتبط» اشتباه نگیرید).
۳. *نادیده گرفتن نویز:* اطلاعاتی که مستقیماً به «سوال کاربر» مربوط نیستند را نادیده بگیرید.
۴. *پایبندی:* اگر پاسخ سوال به هیچ وجه در «زمینه» وجود نداشت، بگویید: "متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم."
"""

# سوال کاربر و زمینه در قالب «انسان» قرار می‌گیرند
HUMAN_TEMPLATE = """
زمینه (متن‌های یافت‌شده از PDF):
{context}
---
سوال کاربر:
{question}
---
پاسخ دقیق و کامل شما (فقط به زبان فارسی):
"""

# ساخت قالب چت نهایی
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_TEMPLATE),
    ("human", HUMAN_TEMPLATE),
])

print("--- ✅ «قالب چت» (Prompt) ترکیبی نهایی با موفقیت ساخته شد ---")

--- 🤖 در حال تعریف «قالب چت» (ChatPromptTemplate) - نسخه ترکیبی نهایی ---
--- ✅ «قالب چت» (Prompt) ترکیبی نهایی با موفقیت ساخته شد ---


In [77]:
# --- دانلود فایل‌های PDF ---
print("--- در حال دانلود فایل‌های PDF ---")
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Phd1405-.konkur.in.pdf" -O daftarche_asli.pdf
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Eslahiye-PHD1405-.konkur.in.pdf" -O daftarche_eslahiye.pdf
print("--- فایل‌ها دانلود شدند ---")

--- در حال دانلود فایل‌های PDF ---
--- فایل‌ها دانلود شدند ---


In [78]:
# --- خواندن، بارگذاری و خُرد کردن متن‌ها ---
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ۱. خواندن هر دو فایل PDF
print("... در حال خواندن فایل‌های PDF")
loader_asli = PyPDFLoader("daftarche_asli.pdf")
loader_eslahiye = PyPDFLoader("daftarche_eslahiye.pdf")
all_pages = loader_asli.load() + loader_eslahiye.load()
print(f"--- مجموعا {len(all_pages)} صفحه بارگذاری شد ---")

# ۲. خُرد کردن متن‌ها به تکه‌های کوچک (Chunks)
print("... در حال خُرد کردن متن‌ها")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
text_documents = text_splitter.split_documents(all_pages)
print(f"--- متن‌ها به {len(text_documents)} تکه (Chunk) تقسیم شدند ---")


... در حال خواندن فایل‌های PDF
--- مجموعا 58 صفحه بارگذاری شد ---
... در حال خُرد کردن متن‌ها
--- متن‌ها به 394 تکه (Chunk) تقسیم شدند ---


In [79]:
# --- ساخت یا بارگذاری «بایگانی هوشمند» (نسخه نهایی و ایمن) ---
from langchain_community.vectorstores import FAISS
import os

# --- تغییر کلیدی: نام فایل حالا شامل نام مدل امبدینگ است ---
DB_FAISS_PATH = "faiss_index_daftarche_labse"
print(f"--- 💾 نام بایگانی هوشمند: {DB_FAISS_PATH} ---")

# --- حذف دستور !rm -rf : ما دیگر فایل را به زور پاک نمی‌کنیم ---

if not os.path.exists(DB_FAISS_PATH):
    print(f"... «بایگانی هوشمند» ({DB_FAISS_PATH}) وجود ندارد. در حال ساخت با مدل (LaBSE)...")

    vectorstore = FAISS.from_documents(
        text_documents,
        embedding=embeddings # از متغیر embeddings (جدید LaBSE) استفاده می‌کند
    )

    print("... در حال ذخیره بایگانی روی دیسک ...")
    vectorstore.save_local(DB_FAISS_PATH)
    print(f"--- «بایگانی هوشمند» جدید با موفقیت ساخته و در {DB_FAISS_PATH} ذخیره شد ---")

else:
    print(f"--- «بایگانی هوشمند» ({DB_FAISS_PATH}) از قبل وجود دارد. در حال بارگذاری سریع از دیسک...")
    vectorstore = FAISS.load_local(
        DB_FAISS_PATH,
        embeddings, # مدل امبدینگ برای بارگذاری لازم است
        allow_dangerous_deserialization=True # برای FAISS در LangChain لازم است
    )
    print("--- «بایگانی هوشمند» با موفقیت بارگذاری شد ---")

--- 💾 نام بایگانی هوشمند: faiss_index_daftarche_labse ---
--- «بایگانی هوشمند» (faiss_index_daftarche_labse) از قبل وجود دارد. در حال بارگذاری سریع از دیسک...
--- «بایگانی هوشمند» با موفقیت بارگذاری شد ---


In [80]:
# --- (ب): ساخت «کتابدار» (Retriever) - با استراتژی Similarity ---

# --- تغییر کلیدی ---
# ما استراتژی "mmr" را حذف کردیم و به حالت پیش‌فرض (similarity) برگشتیم.
# ما اعتماد داریم که "بینی معنی‌فهم" LaBSE آنقدر قوی است
# که در حالت similarity هم نتایج خوبی برگرداند.

retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5} # ۵ نتیجه برتر بر اساس شباهت
)

print("--- «کتابدار» (Retriever) با استراتژی Similarity (پیش‌فرض) آماده جستجو است ---")

--- «کتابدار» (Retriever) با استراتژی Similarity (پیش‌فرض) آماده جستجو است ---


In [82]:
# --- (جدید): ساخت زنجیره RAG با LCEL ---
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("--- 🧠 در حال ساخت زنجیره RAG (RAG Chain) مدرن با LCEL ---")

# این تابع اسناد بازیابی شده را به یک متن واحد تبدیل می‌کند
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# --- ساخت زنجیره (Chain) با استفاده از LCEL (زبان عبارتی LangChain) ---
# این زنجیره به صورت خودکار، ایمن و بهینه تمام مراحل را انجام می‌دهد:
# 1. سوال را می‌گیرد (RunnablePassthrough)
# 2. به کتابدار (retriever) می‌دهد و اسناد را فرمت می‌کند (format_docs)
# 3. دیکشنری {context, question} را به پرامپت می‌دهد
# 4. خروجی پرامپت را به «مغز» (llm) می‌دهد
# 5. خروجی «مغز» را به «تمیزکننده» (StrOutputParser) می‌دهد

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt  # پرامپت «سخت‌گیرانه» که در سلول ۵ ساختیم
    | llm     # مدل gemma-9b
    | StrOutputParser() # پارسر را مستقیماً در زنجیره قرار می‌دهیم
)

print("--- ✅ زنجیره RAG با موفقیت ساخته شد ---")

--- 🧠 در حال ساخت زنجیره RAG (RAG Chain) مدرن با LCEL ---
--- ✅ زنجیره RAG با موفقیت ساخته شد ---


In [83]:
# --- : تعریف تابع 'ask_robot' (نسخه پاک‌سازی شده) ---

print("--- 🤖 در حال تعریف تابع 'ask_robot' (نسخه نهایی) ---")

def ask_robot(question):
    """
    این تابع یک سوال می‌گیرد و آن را مستقیماً به زنجیره RAG می‌دهد.
    (بدون هیچ پرینت اضافی)
    """
    try:
        # --- فراخوانی کل زنجیره RAG ---
        # تمام print های مربوط به دیباگ حذف شدند.
        response = rag_chain.invoke(question)

        return response.strip() # فقط جواب نهایی و تمیز شده

    except Exception as e:
        return f"خطایی در پردازش رخ داد: {e}"

print("--- ✅ تابع 'ask_robot' (نسخه نهایی) با موفقیت ساخته شد ---")

--- 🤖 در حال تعریف تابع 'ask_robot' (نسخه نهایی) ---
--- ✅ تابع 'ask_robot' (نسخه نهایی) با موفقیت ساخته شد ---


In [84]:
# --- (سوال نهایی): اجرای تمام سوالات تستی و فنی ---

print("--- 🤖 شروع تست ربات با سوالات حرفه‌ای 🤖 ---")

# --- دسته ۱: سوالات اصلی شما ---

query_1 = "سهمیه ایثارگران چقدر است؟"
print(f"[شما]: {query_1}")
answer_1 = ask_robot(query_1)
print(f"\n[ربات]:\n{answer_1}")
print("\n" + "="*50)

query_2 = "گرایش های رشته زبان و ادبیات فارسی؟"
print(f"[شما]: {query_2}")
answer_2 = ask_robot(query_2)
print(f"\n[ربات]:\n{answer_2}")
print("\n" + "="*50)

query_3 = "گرایش های علوم اقتصادي؟"
print(f"[شما]: {query_3}")
answer_3 = ask_robot(query_3)
print(f"\n[ربات]:\n{answer_3}")
print("\n" + "="*50)

# --- دسته ۲: سوالات فنی و دقیق (برای نمایش قدرت جستجو) ---

query_4 = "نمره کل نهایی چگونه محاسبه می شود؟"
print(f"[شما]: {query_4}")
answer_4 = ask_robot(query_4)
print(f"\n[ربات]:\n{answer_4}")
print("\n" + "="*50)

query_5 = "ضریب تاثیر سوابق آموزشی، پژوهشی و فناوری چند درصد است؟"
print(f"[شما]: {query_5}")
answer_5 = ask_robot(query_5)
print(f"\n[ربات]:\n{answer_5}")
print("\n" + "="*50)

# --- دسته ۳: سوالات ترکیبی (برای نمایش درک اصلاحیه) ---

query_6 = "کد رشته محل‌های مربوط به رشته مهندسی کامپیوتر - هوش مصنوعی کدامند؟"
print(f"[شما]: {query_6}")
answer_6 = ask_robot(query_6)
print(f"\n[ربات]:\n{answer_6}")
print("\n" + "="*50)

query_7 = "بر اساس اصلاحیه، تاریخ دقیق بررسی سوابق علمی و مصاحبه چه زمانی است؟"
print(f"[شما]: {query_7}")
answer_7 = ask_robot(query_7)
print(f"\n[ربات]:\n{answer_7}")
print("\n" + "="*50)

# --- دسته ۴: سوالات کنترلی (برای نمایش ایمنی ربات) ---

query_8 = "اسم من چیست؟"
print(f"[شما]: {query_8}")
answer_8 = ask_robot(query_8)
print(f"\n[ربات]:\n{answer_8}")
print("\n" + "="*50)

query_9 = "پایتخت ایران کجاست؟"
print(f"[شما]: {query_9}")
answer_9 = ask_robot(query_9)
print(f"\n[ربات]:\n{answer_9}")
print("\n" + "="*50)


print("--- 🏁 تست ربات تمام شد 🏁 ---")


--- 🤖 شروع تست ربات با سوالات حرفه‌ای 🤖 ---
[شما]: سهمیه ایثارگران چقدر است؟

[ربات]:
سهمیه ایثارگران در هر کدرشتهمحل به دو صورت است:

1. 25% از ظرفیت هر کدرشتهمحل به "همسر و فرزندان شهدا و مفقودالاثران"، "آزادگان و همسر و فرزندان آنان" و "جانبازان 52% و بالاتر و همسر و فرزندان آنان" اختصاص دارد.

2. 5% از ظرفیت هر کدرشتهمحل به "جانبازان زیر 52% و همسر و فرزندان آنان" و "همسر و فرزندان رزمندگان با حداقل شش ماه حضور داوطلبانه در جبهه" اختصاص دارد.

[شما]: گرایش های رشته زبان و ادبیات فارسی؟

[ربات]:
گرایش های رشته زبان و ادبیات فارسی عبارتند از:

* **ادبیات حماسي**
* **ادبيات عرفاني**
* **ادبيات غنايي**
* **آموزش زبان فارسي**

[شما]: گرایش های علوم اقتصادي؟

[ربات]:
گرایش  «اقتصاد ایران»

[شما]: نمره کل نهایی چگونه محاسبه می شود؟

[ربات]:
متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم.

[شما]: ضریب تاثیر سوابق آموزشی، پژوهشی و فناوری چند درصد است؟

[ربات]:
متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم.

[شما]: کد رشته محل‌های مربوط به رشته مهندسی کامپیوتر - هوش مصنوعی کدامند؟

[ر